NLP analysis of data in `~/.metha`.
Load just enough data to get something meaningful.

Identifier format before March 2007 differ from the present 

In [56]:
import gzip, glob
from datetime import datetime
import pandas as pd
import nltk
import xml.etree.ElementTree as ET

In [7]:
files = glob.glob('/home/atkm/.metha/20170923/2017-08-31-*.xml.gz')

In [12]:
f = files[0]
with gzip.open(f, 'rb') as f:
    xml = ET.parse(f)

In [17]:
xml.__class__

xml.etree.ElementTree.ElementTree

In [15]:
root = xml.getroot()

In [22]:
root.getchildren()

[<Element 'responseDate' at 0x7f162cce4728>,
 <Element 'request' at 0x7f162cce43b8>,
 <Element 'error' at 0x7f162cce4908>,
 <Element 'GetRecord' at 0x7f162cce4ae8>,
 <Element 'Identify' at 0x7f162cce4c28>,
 <Element 'ListIdentifiers' at 0x7f162cce4c78>,
 <Element 'ListMetadataFormats' at 0x7f162cce4cc8>,
 <Element 'ListRecords' at 0x7f162cce4d18>,
 <Element 'ListSets' at 0x7f162c6eba48>]

In [27]:
for c in root.getchildren():
    if c.tag != 'ListRecords': # ListRecords contains all documents
        print(c.tag, ' ! ', c.getchildren())

responseDate  !  []
request  !  []
error  !  []
GetRecord  !  [<Element 'record' at 0x7f162cce49f8>]
Identify  !  []
ListIdentifiers  !  []
ListMetadataFormats  !  []
ListSets  !  []


In [33]:
# nothing in root.GetRecord
for c in root.find('GetRecord').iter():
    print(c.tag, ' ! ', c.attrib)

GetRecord  !  {}
record  !  {}
header  !  {'status': ''}
metadata  !  {}
about  !  {}


[]

In [84]:
a.append(1)

In [85]:
a,b,c

([1], [1], [1])

In [51]:
xml.getroot().find('ListRecords').find('record').find('metadata').find('arXiv')

In [89]:
def xml_to_df(xml):
    """
    xml: ElementTree such that xml.getroot() returns an Element
    """
    root = xml.getroot()
    #OAIurl = '{http://www.openarchives.org/OAI/2.0/}'
    arXiv_prefix = '{http://arxiv.org/OAI/arXiv/}'
    title, abstract, authors, created, categories, identifier = ([] for i in range(6))
    # Use Element.iter() ? (depth-first iteration of tree)
    for record in root.find('ListRecords').findall("record"):        
        metadata = record.find('metadata').find(arXiv_prefix + "arXiv")
        c = metadata.find(arXiv_prefix + "created").text
        created.append( datetime.strptime(c, "%Y-%m-%d") )
        authors.append( metadata.find(arXiv_prefix + 'authors').text )
        categories.append( metadata.find(arXiv_prefix + "categories").text )
        title.append( metadata.find(arXiv_prefix + "title").text )
        identifier.append( metadata.find(arXiv_prefix + "id").text )
        abstract.append( metadata.find(arXiv_prefix + "abstract").text.strip() )

        #arxiv_id = record.find(OAIurl + 'header').find(OAIurl + 'identifier')
        #datestamp = record.find(OAIurl+'header').find(OAIurl+'datestamp')
        #datestamp = datetime.strptime(datestamp.text, '%Y-%m-%d')
        """ Don't think DOI is needed
        # if there is more than one DOI use the first one
        # often the second one (if it exists at all) refers
        # to an eratum or similar
        doi = info.find(arXivOAI+"doi")
        if doi is not None:
            doi = doi.text.split()[0]
        """

    # Much faster than appending each record as a Series to a df
    df = pd.DataFrame({'title': title,
                    'abstract': abstract,
                    'authors': authors,
                    'created': created,
                    'categories': categories,
                    'id': identifier})
    return df

In [90]:
df = xml_to_df(xml)

In [91]:
df.shape

(1000, 6)

In [98]:
df['created'].describe()

count                    1000
unique                    157
top       2017-07-31 00:00:00
freq                      109
first     2017-02-21 00:00:00
last      2017-07-31 00:00:00
Name: created, dtype: object